<p><strong><font size="6">WALOUS</font></strong></p>

<p><strong><font size="6">Fusion LC classifications results</font></strong></p>

<p><strong><font size="6">Postprocessing - Resampling 1m Vectorisation etc..</font></strong></p>

WALOUS_OCS_FUSION - Copyright (C) <2020> <Service Public de Wallonie (SWP), Belgique,
					          		Institut Scientifique de Service Public (ISSeP), Belgique,
									Université catholique de Louvain (UCLouvain), Belgique,
									Université Libre de Bruxelles (ULB), Belgique>
						 							
	
List of the contributors to the development of WALOUS_OCS_FUSION: see LICENSE file.
Description and complete License: see LICENSE file.
	
This program (WALOUS_OCS_FUSION) is free software: 
you can redistribute it and/or modify it under the terms of the 
GNU General Public License as published by the Free Software 
Foundation, either version 3 of the License, or (at your option) 
any later version.

This program is distributed in the hope that it will be useful,
but WITHOUT ANY WARRANTY; without even the implied warranty of
MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
GNU General Public License for more details.

You should have received a copy of the GNU General Public License
along with this program (see COPYING file).  If not, 
see <http://www.gnu.org/licenses/>.

This python code implement the method developed by ANAGEO (ULB). 

Code developped on Linux Mint 18.1 (Ubuntu Xenial 16.04) and GRASS GIS 7.3.svn (r71315).

# Table of Contents

<div id="toc"></div>

The following cell is a Javascript section of code for building the Jupyter notebook's table of content.

In [1]:
%%javascript
$.getScript('https://kmahelona.github.io/ipython_notebook_goodies/ipython_notebook_toc.js')

<IPython.core.display.Javascript object>

# Define working environment

**Import libraries**

In [2]:
# Import libraries needed for setting parameters of operating system 
import os
import sys
import csv
import tempfile
import subprocess
import glob
import pickle
print(sys.version)

2.7.14 (default, Oct 12 2017, 15:50:02) [GCC]


In [3]:
## Import multiprocessing and functools libraries
import multiprocessing
from multiprocessing import Pool
from functools import partial

** Add folder with SCR provided belong to this notebook**

In [4]:
# Add local module to the path
src = os.path.abspath('/export/miro/cbassine/fusion/RF_fusion/WALOUS-master/SRC')
if src not in sys.path:
    sys.path.append(src)

** Setup environment variables for TAIS DESKTOP (Linux Mint + GRASS Dev) **

Please edit the file in `../SRC/config.py`, containing the configuration parameters, according to your own computer setup. The following cell is used to run this file.



In [60]:
run /export/miro/cbassine/fusion/RF_fusion/WALOUS-master/SRC/config_postprocess.py

In [61]:
print config_parameters

{'list_tiles': '/export/miro/cbassine/fusion/RF_fusion/Postprocess6/list_tiles', 'outputfolder_Logfile': '/export/miro/cbassine/fusion/RF_fusion/Postprocess6/Log_file', 'permanent_mapset': 'PERMANENT', 'GRASSBIN': '/usr/bin/grass78', 'locationepsg': '31370', 'outputfolder': '/export/miro/cbassine/fusion/RF_fusion/Postprocess6', 'outputfolder_Vecteur': '/export/miro/cbassine/fusion/RF_fusion/Postprocess6/Vecteur', 'gisdb': '/export/miro/cbassine/GRASSDATA', 'location': 'WALOUS_31370', 'PYTHONLIB': '/usr/bin/python2.7', 'njobs': 6, 'outputfolder_Raster': '/export/miro/cbassine/fusion/RF_fusion/Postprocess6/Raster', 'GISBASE': '/usr/lib64/grass78'}


In [7]:
os.system("module load gdal/2.2.4")

0

In [8]:
# Import functions that setup the environmental variables
import environ_variables as envi

In [9]:
# Set environmental variables
envi.setup_environmental_variables() 
# Display current environment variables of your computer
envi.print_environmental_variables()

LESS = -M -I -R 	
BASH_ENV = /usr/share/lmod/7.6.1/init/bash 	
CPU = x86_64 	
BASH_FUNC_module%% = () {  eval $($LMOD_CMD bash "$@") && eval $(${LMOD_SETTARG_CMD:-:} -s sh)
} 	
SSH_CONNECTION = 10.36.1.8 57890 10.36.22.8 22 	
SHELL = /bin/bash 	
XDG_DATA_DIRS = /usr/share 	
HISTSIZE = 1000 	
_ModuleTable003_ = aSIsfSx9LG1wYXRoQT17Ii91c3Ivc2hhcmUvbW9kdWxlcyIsIi91c3Ivc2hhcmUvbG1vZC9tb2R1bGVmaWxlcyIsfSxbInN5c3RlbUJhc2VNUEFUSCJdPSIvdXNyL3NoYXJlL2xtb2QvbW9kdWxlZmlsZXMiLH0= 	
LESS_ADVANCED_PREPROCESSOR = no 	
MANPATH = /usr/lib64/mpi/gcc/openmpi/share/man:/usr/local/man:/usr/local/share/man:/usr/share/man 	
JAVA_HOME = /usr/lib64/jvm/jre-1.8.0-openjdk 	
PROFILEREAD = true 	
XDG_RUNTIME_DIR = /run/user/49919 	
JPY_PARENT_PID = 1463 	
PYTHONPATH = /export/apps/os151/OTB-7.0.0/lib/otb/python:/export/apps/os151/gdal-3.0.1/gdal/lib64/python3.6/site-packages:/usr/lib64/grass78/etc/python:/usr/lib64/grass78/etc/python/grass:/usr/lib64/grass78/etc/python/grass/script 	
GISRC = /export/homes/cbassine

** GRASS GIS Python libraries **

In [10]:
from grass_session import Session
# Import libraries needed to launch GRASS GIS in the jupyter notebook
import grass.script.setup as gsetup
# Import libraries needed to call GRASS using Python
import grass.script as gscript

GRASSBIN: /usr/bin/grass78
GISBASE: /usr/lib64/grass78


** Other functions**

In [11]:
# Import function that check existance and create GRASS GIS database folder if needed
from grass_database import check_gisdb, check_location, check_mapset, working_mapset
# Import functions for processing time information
from processing_time import start_processing, print_processing_time
# Import function that generate a random name in the GRASS GIS environement
from random_layer_name import random_layer_name
# Import function that check and create folder
from mkdir import check_create_dir
# Import function that check if GRASS GIS add-on is installed and install it if needed
from gextension import check_install_addon
# Import function for .gzip archive management
from gzip_management import decompress_gzip
# Import function for computation of proportion of categorical raster in zones (segments)
from compute_proportion_categorical import CategoStats
# Import function for getting list of class of a raster
from data_prep import data_prep
# Import function the allow sorting strings with number as if they was number (natural order)
from sorting_natural import natural_keys
# Import function that compute ERP on csv file
from EquivalentReferenceProbability import ComputeERPfromCsv
# Import function that replace "empty" values from output .csv to zero
from CsvEmptyValues import CsvChangeEmptyByO

**Import libraries**

In [12]:
# Import function that check and create folder
from mkdir import check_create_dir

In [13]:
# Import function that generate a random name in the GRASS GIS environement
from random_layer_name import random_layer_name

## Special functions

In [14]:
# Import function that check existance and create GRASS GIS database folder if needed
from grass_database import check_gisdb, check_location, check_mapset, working_mapset
# Import functions for processing time information
from processing_time import start_processing, print_processing_time

In [18]:
def launch_mapset(mapset):
    #Declare empty list that will contain the messages to return
    return_message = []
    # Check if the location exists and create it if not, with the CRS defined by the epsg code 
    return_message.append(check_location(config_parameters["gisdb"],config_parameters['location'],config_parameters["locationepsg"]))
    # Check if mapset exists
    return_message.append(check_mapset(config_parameters["gisdb"],config_parameters['location'],mapset))
    # Change the current working GRASS GIS session mapset
    return_message.append(working_mapset(config_parameters["gisdb"],config_parameters['location'],mapset))
    # Check if a DB connexion exist and create it if not exists
    gscript.run_command('db.connect', quiet=True, flags='c')
    # Return
    return return_message

In [19]:
def GetMapsetsAccess(list_mapsets):
    #Declare empty string that will contain the messages to return
    return_message = ''
    try:
        # Add mapsets with input data to the GRASS GIS research path
        for mapset in list_mapsets:
            gscript.run_command('g.mapsets', quiet=True, mapset=mapset, operation="add")
        return_message = "Access to other mapset added"
    except:
        return_message += "ERROR: Add access to other Mapsets failed. Please check for problem."
    return return_message

In [20]:
def DefineComputationRegionAndMask(tile_cat,resamp_resolution='1', grow_mmu='16'):
    #Declare empty string that will contain the messages to return
    return_message = ''
    try:
        return_message = "Working on tile '%s'\n"%tile_cat
        # 'Grow' parameter is used to expend the computational region ouside of the current tile to deal with bordering objects smaller than MMU
        gscript.run_command('g.region', flags='a', raster='RF_fusion_tile_%s'%tile_cat, 
                            res=resamp_resolution, grow=grow_mmu)
        # Define MASK for clipping at the end
        gscript.run_command('r.mask', overwrite=True, quiet=True, raster='RF_fusion_tile_%s'%tile_cat) 
        gscript.run_command('g.copy', overwrite=True, quiet=True, raster='MASK,MASK_tile25') #25cm resolution mask following the original limit of the tile
        gscript.run_command('r.mask', quiet=True, flags='r') #Remove mask
        # Define MASK for processing (overlapping with the neighboring tiles)
        gscript.run_command('r.buffer', overwrite=True, quiet=True, input='MASK_tile25', 
                            output='MASK_tmp', distances=grow_mmu)
        rulefile = gscript.tempfile()
        with open(rulefile, 'w') as f:
            f.write('2 = 1\n* = *') #Reclass rule
        gscript.run_command('r.reclass', overwrite=True, quiet=True, input='MASK_tmp', output='MASK_processing', rules=rulefile)
        # Print
        return_message += "--> Setting of computational region and masks succeeded."
    except:
        return_message += "ERROR: Setting of computional region and masks failed for cutline '%s'. Please check for problem."%tile_cat
    return return_message

In [21]:
def Resamp(tile_cat):
    #Resample classification 
    return_message = ''
    try:
        gscript.run_command('r.mask', overwrite=True, quiet=True, raster='MASK_processing') 
        gscript.run_command('r.resamp.stats', overwrite=True, quiet=True, input='fusion_lc', 
                            output='resamp', method='mode')
        #Ensure for CEL type by creating a new map using r.mapcalc
        gscript.mapcalc('resamp_tmp=int(resamp)', overwrite=True, quiet=True) 
        gscript.run_command('g.rename', overwrite=True, quiet=True, raster='resamp_tmp,resamp')
        #Make a copy of the resampled map, nammed ' that will be used as first tmp map for the reclassification
        gscript.run_command('g.copy', overwrite=True, quiet=True, raster='resamp,reclass_tmp')
        return_message += "--> Resampling succeeded."
    except:
        return_message += "ERROR: Resampling for tile '%s' failed. Please check for problem."%tile_cat
    return return_message   

In [22]:
def GetForestryMask(tile_cat, schrink_growth_radius, overwrite=True):
    #Create forest mask
    return_message = ''
    try:
        #Initial forest mask (class 8 and 9 dans la fusion)
        formula = "forestmask_binary=if((resamp==8||resamp==9),1,null())"
        gscript.mapcalc(formula, overwrite=overwrite, quiet=True)
        #Moving window modal filter
        gscript.run_command('r.neighbors', overwrite=True, quiet=True, input='forestmask_binary',
                            output='forestmask_binary_neighbors', method='mode', size='5')
        gscript.run_command('g.rename', overwrite=True, quiet=True, raster='forestmask_binary_neighbors,forestmask_binary')
        #Schrinking
        gscript.run_command('r.grow', overwrite=True, quiet=True, input='forestmask_binary',
                            output='forestmask_binary_shrink', radius='-%s'%schrink_growth_radius, old='1', new='1')
        #Growing
        gscript.run_command('r.grow', overwrite=True, quiet=True, input='forestmask_binary_shrink',
                            output='forestmask_binary_grow', radius='%s'%schrink_growth_radius, old='1', new='1')
        gscript.run_command('g.remove', flags='f', quiet=True, type='raster', name='forestmask_binary_shrink')
        gscript.run_command('g.rename', overwrite=True, quiet=True, raster='forestmask_binary_grow,forestmask_binary')
        # Print
        return_message += "--> Creation of forest mask succeeded."
    except:
        return_message += "ERROR: Creation of forest mask failed for tile '%s' failed. Please check for problem."%tile_cat
    return return_message   

In [23]:
""" 
Function to generate and execute an SQL insert query based on input 'table_name','head' and 'value_dict'. 
The key of the dictionnary 'value_dict' should will be the first column of the table (unique id) and the values should
contain a list with the values (the other columns). Please be sure that len('head') == 1+len(value_dict[key])
"""
def SqlInsert(table_name, header, value_dict, overwrite=True):
    sql_query = gscript.tempfile()
    fsql = open(sql_query, 'w')
    fsql.write('BEGIN TRANSACTION;\n')
    if gscript.db_table_exist(table_name):
        if overwrite:
            fsql.write('DROP TABLE %s;\n' % table_name)
        else:
            gscript.fatal(_("Table %s already exists. Use 'overwrite=True' to overwrite" % table_name))
    create_statement = 'CREATE TABLE ' + table_name + ' (cat int PRIMARY KEY);\n'
    fsql.write(create_statement)
    for col in header[1:]:
        if col.split('_')[-1] == 'cat':  # Mode column should be integer
            addcol_statement = 'ALTER TABLE %s ADD COLUMN %s integer;\n' % (table_name, col)
        else: # Proportions column should be double precision
            addcol_statement = 'ALTER TABLE %s ADD COLUMN %s double precision;\n' % (table_name, col)
        fsql.write(addcol_statement)
    for key in value_dict:
            insert_statement = 'INSERT INTO %s VALUES (%s, %s);\n' % (table_name, key, ','.join([str(x) for x in value_dict[key]]))
            fsql.write(insert_statement)
    fsql.write('END TRANSACTION;')
    fsql.close()
    gscript.run_command('db.execute', input=sql_query, quiet=True)

'''
Function to compute shape statistics for each clump and import it as table in GRASS GIS
'''
def GetClumpShapeStatistics(overwrite=True):
    # Path to temporary file for 'r.object.geometry' output
    tmp_csv = "%s_robjectgeometry"%gscript.tempfile() #definit tmp_csv comme un fichier temporaire
    # Compute shape statistics for each clump
    gscript.run_command('r.object.geometry', overwrite=True, flags='m', input='clump', output=tmp_csv) #calculates geometry parameters for raster objects
    ### clump from "r.clump" recategorizes data in a raster map by grouping cells that form physically discrete areas into unique categories
    # Load csv content in python dictionnary
    incsv = open(tmp_csv, 'r')
    reader = csv.reader(incsv, delimiter='|')
    header = reader.next()
    value_dict = {row[0]:row[1:] for row in reader}
    incsv.close()
    # Insert SQL
    table_name = "geom"
    SqlInsert(table_name, header, value_dict, overwrite=True)
    
'''
Function to retrieve the land cover class of each clump and import it as table in GRASS GIS
'''
#recuperation de la classe de LC pour chaque groupe (clump) 
def GetClumpLCClass(overwrite=True):
    # Path to temporary file for 'r.object.geometry' output
    tmp_csv = "%s_runivar"%gscript.tempfile() 
    # Compute r.univar to get class label of the clump
    gscript.run_command('r.univar', overwrite=True, quiet=True, flags='t', map='reclass_tmp', zones='clump', output=tmp_csv)
    ## r.univar = calculates univariate statistics from the non-null cells of a raster map, flag -t for a table output
    # Load csv content in python dictionnary
    incsv = open(tmp_csv, 'r')
    reader = csv.reader(incsv, delimiter='|')
    reader.next() # Skip the first row (header)
    header = ["cat","label"] # Create a header with custom column name
    value_dict = {row[0]:row[4:5] for row in reader} #Take only the column corresponding to 'min'
    incsv.close()
    # Insert SQL
    table_name = "label"
    SqlInsert(table_name, header, value_dict, overwrite)

'''
Function to know information on each clump to know if it belong to the forestry mask
'''
def GetForestMaskInfo(overwrite=True):
    # Path to temporary file for 'r.object.geometry' output
    #tmp_csv = "%s_rzonalclasses_forest"%gscript.tempfile()
    tmp_csv = "%s_rzonalcount_forest"%gscript.tempfile()
    #gscript.run_command('r.zonal.classes', flags='n', overwrite=True, quiet=True,
                        #zone_map='clump', raster='forestmask_binary', statistics='mode', csvfile=tmp_csv)
    gscript.run_command('r.stats.zonal', overwrite=True, quiet=True,
                        cover='forestmask_binary', base='clump', method='count', output='rzonal_sum')
    gscript.run_command('r.stats.zonal', overwrite=True, quiet=True,
                        cover='clump', base='clump', method='count', output='rzonal_count')
    #formula = "proportion=if(((rzonal_sum/rzonal_count)>=0.5),1,null())"
    formula= "proportion=(float(rzonal_sum)/float(rzonal_count)>=0.5?1:0)"
    gscript.run_command('r.mapcalc',expression=formula, overwrite=overwrite, quiet=True)
    gscript.run_command('r.univar', overwrite=True, quiet=True, flags='t', map='proportion', zones='clump', output=tmp_csv)
    ## r.zonal.classes = calculates zonal classes proportion describing raster areas's composition
    # Load csv content in python dictionnary
    incsv = open(tmp_csv, 'r')
    #incsv_count = open(tmp_csv_count, 'r')
    reader = csv.reader(incsv, delimiter='|')
    reader.next() # Skip the first row (header)
    header = ["cat","forest_mask"] # Create a header with custom column name
    #value_dict = {row[0]:(row[1] if row[1]!="NULL" else '0') for row in reader} #Take only the column corresponding to 'min'    #value_dict = {row[0]:row[2] if row[2]>0.5 else '0' for row in reader_count}
    value_dict={row[0]:row[4] for row in reader}
    incsv.close()
    #Insert SQL
    table_name = "forestry_mask"
    SqlInsert(table_name, header, value_dict, overwrite)

'''
Function to retrieve the neighborhood of each clump and import it as table in GRASS GIS
'''
def GetClumpNeighbor(overwrite=True):
    # Path to temporary file for 'r.object.geometry' output
    tmp_csv = "%s_neighbor"%gscript.tempfile()
    # Compute neighborhood matrix
    gscript.run_command('r.neighborhoodmatrix', overwrite=True, flags='l', input='clump', output=tmp_csv) #Calculates geometry parameters for raster objects, -l length of common border
    #r.neighborhoodmatrix identifies all adjacency relations between objects 
    #(aka segments or clumps identified by identical integer cell values of adjacent pixels) 
    # in a raster map and exports these as a 2xn matrix where n is the number of neighborhood relations 
    # with each relation listed in both directions (i.e. if a is neighbor of b, the list will contain a,b and b,a). 
    # If a path to an output file is specified, the matrix will be written to that file, 
    #otherwise it will be sent to standard output.
    
    # Load csv content in python dictionnary
    incsv = open(tmp_csv, 'r')
    reader = csv.reader(incsv, delimiter='|')
    header = ["cat","sega","segb","border_count"] # Create a header
    value_dict = {id_:row[:] for id_,row in enumerate(reader)}
    incsv.close()
    # Insert SQL
    table_name = "rmatrix"
    SqlInsert(table_name, header, value_dict, overwrite)

""" 
Function to pivot rmatrix table to horizontal table with proportion to each class
"""
def GetNeighborClassProportion(overwrite=True):
    global labels_list #fction qui permet de modifier une variable depuis le corps d'une fonction
    ### Table 'tmp_a'
    table_name = "tmp_a"
    sql_query = gscript.tempfile()
    fsql = open(sql_query, 'w')
    fsql.write('BEGIN TRANSACTION;\n')
    if gscript.db_table_exist(table_name):
        if overwrite:
            fsql.write('DROP TABLE %s;\n'%table_name)
        else:
            gscript.fatal(_("Table %s already exists. Use 'overwrite=True' to overwrite"%table_name))
    create_statement = 'CREATE TABLE %s AS '%table_name
    create_statement += 'SELECT a.sega, b.label, border_count FROM rmatrix AS a '
    create_statement += 'LEFT JOIN label AS b ON a.segb=b.cat;\n'
    fsql.write(create_statement)
    fsql.write('END TRANSACTION;')
    fsql.close()
    gscript.run_command('db.execute', input=sql_query, quiet=True)
    
    ### Table 'tmp_b'
    table_name = "tmp_b"
    sql_query = gscript.tempfile()
    fsql = open(sql_query, 'w')
    fsql.write('BEGIN TRANSACTION;\n')
    if gscript.db_table_exist(table_name):
        if overwrite:
            fsql.write('DROP TABLE %s;\n'%table_name)
        else:
            gscript.fatal(_("Table %s already exists. Use 'overwrite=True' to overwrite"%table_name))
    create_statement = 'CREATE TABLE %s AS WITH '%table_name
    create_statement += 'border_lenght AS (SELECT sega, sum(border_count) AS sum_border FROM tmp_a GROUP BY sega),'
    create_statement += 'tempotable AS (SELECT a.sega, a.label, round((a.border_count*1.0/b.sum_border*1.0),8) AS percent_border FROM tmp_a AS a LEFT JOIN border_lenght AS b ON a.sega=b.sega)'
    create_statement += 'SELECT sega AS seg, label, sum(percent_border) AS percent_border FROM tempotable GROUP BY sega, label ORDER BY sega, label;\n'
    create_statement += 'UPDATE %s SET percent_border=1.0 WHERE percent_border>0.9999;\n'%table_name
    create_statement += 'UPDATE %s SET percent_border=0.0 WHERE percent_border<0.0001;\n'%table_name
    fsql.write(create_statement)
    fsql.write('END TRANSACTION;')
    fsql.close()
    gscript.run_command('db.execute', input=sql_query, quiet=True)
    
    ### Table 'pivot_prop_label'
    table_name = "pivot_prop_label"
    # Get distinct values for 'label' class
    distinctlabelquery="SELECT DISTINCT label FROM tmp_b ORDER BY 1"
#    labels_list = [x[0] for x in gscript.db_select(distinctlabelquery)] #According to classes that exist in the table
    labels_list = ['1','2','21','3','4','5','6','7','8','9','80','90','202'] #Forcing according to classes that should be included in the table
    # Declaration of colums for the pivot table
    columns=["seg INTEGER",]
    [columns.append("prop_%s NUMERIC"%label) for label in labels_list]
    # Crosstab query argument
    crosstabquery = "SELECT seg, label, percent_border FROM tmp_b ORDER  BY 1,2"
    # Built complete sql query
    sql_query = gscript.tempfile()
    fsql = open(sql_query, 'w')
    fsql.write('BEGIN TRANSACTION;\n')
    if gscript.db_table_exist(table_name):
        if overwrite:
            fsql.write('DROP TABLE %s;\n'%table_name)
        else:
            gscript.fatal(_("Table %s already exists. Use 'overwrite=True' to overwrite"%table_name))
    # Create statement for the pivot table
    pivot_statement = "SELECT seg, "
    pivot_statement += ', '.join(["SUM(CASE WHEN label = '%s' THEN percent_border END) AS prop_%s"%(cl,cl) for cl in labels_list])
    pivot_statement += " FROM tmp_b GROUP BY seg"
    create_statement = "CREATE TABLE %s AS %s;\n"%(table_name,pivot_statement)
    fsql.write(create_statement)
    # List of update queries to update each column
    [fsql.write("UPDATE %s SET prop_%s=0.0 WHERE prop_%s is null;\n"%(table_name,label,label)) for label in labels_list]
    fsql.write('END TRANSACTION;')
    fsql.close()
    gscript.run_command('db.execute', input=sql_query, quiet=True)

""" 
Function to 
"""

def AddLabelNthNeighbor(tile_cat, overwrite=True):
    # The trick was found here: https://stackoverflow.com/questions/8436919/second-maximum-and-minimum-values
    # To be able to run the command directly in GRASS, be sure the version of SQlite used in compilation is >3.25
    # The function used here is available only for recent version https://stackoverflow.com/questions/50332436/syntax-error-when-using-row-number-in-sqlite3r
    # Get the largest, second largest and third largest value
    ### SQL query
    table_name = "rank_prop"
    sql_query = gscript.tempfile()
    fsql = open(sql_query, 'w')
    fsql.write('BEGIN TRANSACTION;\n')
    if gscript.db_table_exist('tmp_c'):
        if overwrite:
            fsql.write("DROP TABLE tmp_c;\n")
        else:
            gscript.fatal(_("Table 'tmp_c' already exists. Use 'overwrite=True' to overwrite"))
    subquery = "SELECT seg, percent_border, row_number() OVER ( "
    subquery += "PARTITION BY seg order by percent_border DESC) as rnDesc FROM tmp_B"
    create_statement = "CREATE TABLE tmp_c AS "
    create_statement += 'SELECT seg, '
    create_statement += 'MAX(CASE when rnDesc = 1 THEN percent_border END) as first_prop, '
    create_statement += 'MAX(CASE when rnDesc = 2 THEN percent_border END) as second_prop, '
    create_statement += 'MAX(CASE when rnDesc = 3 THEN percent_border END) as third_prop '
    create_statement += 'from(%s) as SubQueryAlias GROUP BY seg;\n'%subquery
    fsql.write(create_statement)

    if gscript.db_table_exist(table_name):
        if overwrite:
            fsql.write("DROP TABLE %s;\n"%table_name)
        else:
            gscript.fatal(_("Table '%s' already exists. Use 'overwrite=True' to overwrite"%table_name))
    create_statement = "CREATE TABLE %s AS "%table_name
    create_statement += 'SELECT a.*,b.first_prop,b.second_prop,b.third_prop FROM pivot_prop_label AS a LEFT JOIN tmp_C AS b ON a.seg=b.seg;\n'
    fsql.write(create_statement)
    alter_statement = "ALTER TABLE %s ADD COLUMN first_label integer;\n"%table_name
    alter_statement += "ALTER TABLE %s ADD COLUMN second_label integer;\n"%table_name
    alter_statement += "ALTER TABLE %s ADD COLUMN third_label integer;\n"%table_name
    fsql.write(alter_statement)   
    update_queries = []
    for label in labels_list:
        update_queries.append("UPDATE {t} SET first_label={l} WHERE first_prop=prop_{l}".format(t=table_name, l=label))
        update_queries.append("UPDATE {t} SET second_label={l} WHERE second_prop=prop_{l}".format(t=table_name, l=label))
        update_queries.append("UPDATE {t} SET third_label={l} WHERE third_prop=prop_{l}".format(t=table_name, l=label))
    fsql.write(";\n".join(update_queries))
    fsql.write(";\n")
    fsql.write('END TRANSACTION;')
    fsql.close()    
    # Bash file
    bash_sqlite = gscript.tempfile()
    bash = open(bash_sqlite, 'w')
    bash.write('sqlite3 /export/miro/cbassine/GRASSDATA/WALOUS_31370/%s/sqlite/sqlite.db < %s'%(tile_cat,sql_query))
    bash.close() 
    try:
        subprocess.check_call(['bash', bash_sqlite], stderr=subprocess.STDOUT, )
    except subprocess.CalledProcessError:
        message =  "There was an error in the execution of the bash script.\nPlease check and fix."

""" 
Function to join all information together
"""
def GetFinalJoinedTable(overwrite=True):
    ### Table 'pivot_final'
    table_name = "pivot_final"
    sql_query = gscript.tempfile()
    fsql = open(sql_query, 'w')
    fsql.write('BEGIN TRANSACTION;\n')
    if gscript.db_table_exist(table_name):
        if overwrite:
            fsql.write('DROP TABLE %s;\n'%table_name)
        else:
            gscript.fatal(_("Table %s already exists. Use 'overwrite=True' to overwrite"%table_name))
    create_statement = 'CREATE TABLE %s AS '%table_name
    create_statement += 'SELECT a.cat AS seg, a.label, g.area, f.forest_mask, p.* FROM label AS a '
    create_statement += 'LEFT JOIN geom AS g ON a.cat=g.cat '
    create_statement += 'LEFT JOIN forestry_mask AS f ON a.cat=f.cat '
    create_statement += 'LEFT JOIN rank_prop AS p ON a.cat=p.seg ;\n'
    fsql.write(create_statement)
    fsql.write("ALTER TABLE %s ADD COLUMN reclass integer;\n"%table_name)   
    fsql.write('END TRANSACTION;')
    fsql.close()
    gscript.run_command('db.execute', input=sql_query, quiet=True)

""" 
Function to remove all intermediate tables
"""
def RemoveIntermediateTables(overwrite=True):
    list_tables_to_remove = ['label','forestry_mask','geom','rmatrix','tmp_a','tmp_b','tmp_c','pivot_prop_label','rank_prop']
    sql_query = gscript.tempfile()
    fsql = open(sql_query, 'w')
    fsql.write('BEGIN TRANSACTION;\n')
    for table_name in list_tables_to_remove:
        if gscript.db_table_exist(table_name):
            if overwrite:
                fsql.write('DROP TABLE %s;\n'%table_name)
            else:
                gscript.fatal(_("Table %s already exists. Use 'overwrite=True' to overwrite"%table_name))
    fsql.write('END TRANSACTION;')
    fsql.close()
    gscript.run_command('db.execute', input=sql_query, quiet=True)
    
""" 
Wrapper function that create clump and compute neighborhood information for reclassification using rules
"""
def GetNeighborStat(tile_cat, overwrite=True):
    # Define region based on 1m resampled LC
    gscript.run_command('g.region', raster='reclass_tmp')
    # Clump to get unique ID for each LC patch
    gscript.run_command('r.clump', overwrite=True, quiet=True,
                        flags='d', input='reclass_tmp', output='clump')  # flag '-d' to clump also diagonal cells
    GetClumpShapeStatistics(overwrite=True)
    GetClumpLCClass(overwrite=True)
    GetForestMaskInfo(overwrite=True)
    GetClumpNeighbor(overwrite=True)
    GetNeighborClassProportion(overwrite=True)
    AddLabelNthNeighbor(tile_cat, overwrite=True)
    GetFinalJoinedTable(overwrite=True)
    RemoveIntermediateTables(overwrite=True)

In [24]:
def ReclassAccordingToRule(tile_cat, list_case_statement, intermediate=True, overwrite=True):
    #Reclassify clump layer according to CASE statements 
    return_message = ''    
    try:
        #Recompute clumps and neighborhood information
        GetNeighborStat(tile_cat, overwrite=True)
        #Update SQLite table with reclass column 
        return_message = ''
        table_name = "pivot_final"
        if not gscript.db_table_exist(table_name):
            gscript.fatal(_("Table '%s' already exists. Use 'overwrite=True' to overwrite"%table_name))
        sql_query = gscript.tempfile()
        fsql = open(sql_query, 'w')
        fsql.write('BEGIN TRANSACTION;\n')
        fsql.write("UPDATE {t} SET reclass = NULL;\n".format(t=table_name))
        for case_statement in list_case_statement:
            fsql.write("UPDATE {t} SET reclass = ({case});\n".format(t=table_name, case=case_statement))
        fsql.write("UPDATE {t} SET reclass = label WHERE reclass IS NULL;\n".format(t=table_name))
        fsql.write('END TRANSACTION;')
        fsql.close() 
        gscript.run_command('db.execute', input=sql_query, quiet=True)
        #Create r.reclass rule file and reclass the clump map.
        query = "SELECT seg, reclass FROM pivot_final"
        reclass_rule = gscript.tempfile()
        reclass = open(reclass_rule, 'w')
        [reclass.write("%s = %s\n"%(x[0],x[1])) for x in gscript.db_select(query)]
        reclass.close()
        #r.reclass
        if intermediate:
            output_name = 'reclass_tmp'
        else:
            output_name = 'resamp_mmu'
        gscript.run_command('r.reclass', overwrite=True, input='clump', output=output_name, rules=reclass_rule)
        #Create a 'hard' copy of the reclassed raster
        gscript.mapcalc('%s_tmp=%s'%(output_name,output_name),overwrite=True,quiet=True)
        gscript.run_command('g.rename',overwrite=True,quiet=True,raster='%s_tmp,%s'%(output_name,output_name))
        gscript.run_command('r.out.gdal', overwrite=True, input='%s'%(output_name), output='/export/miro/cbassine/tmp/%s.tif'%(output_name), format='GTiff', createopt='COMPRESS=DEFLATE')
        # Print
        return_message += "--> Reclassification of clumps succeeded."
    except:
        return_message += "ERROR: Reclassification of clumps failed for cutline '%s'. Please check for problem."%tile_cat
    return return_message    


In [25]:
def ClipToTileBorder(tile_cat, rasterlist):
    #Clipping raster to match tile's border 
    return_message = ''
    for rastername in rasterlist:
        try:
            gscript.run_command('r.mask', overwrite=True, quiet=True, raster='MASK_tile25') 
            gscript.run_command('r.clip', overwrite=True, quiet=True,
                                input='%s'%rastername, output='tmp_%s'%rastername)
            gscript.run_command('r.mask', quiet=True, flags='r') 
            gscript.run_command('g.rename', overwrite=True, quiet=
                                True, raster='tmp_%s,%s'%(rastername,rastername))
            return_message += "--> Clip raster '%s' to correspond to tile border."%rastername
        except:
            return_message += "ERROR: Clipping raster '%s' to tile border failed for cutline '%s'. Please check for problem."%(rastername,tile_cat)
        if rastername != rasterlist[-1]: return_message += "\n"
    return return_message   

In [26]:
def ColorizeRaster(tile_cat, rasterlist):
    #Declare empty string that will contain the messages to return
    return_message = ''
    for rastername in rasterlist:
        try:
            gscript.run_command('r.colors', map=rastername, rules=data['color_file'])
            return_message += "--> Application of colors table succeeded."
        except:
            return_message += "ERROR: Application of colors table on raster failed for tile %s. Please check for problem."%tile_cat
        if rastername != rasterlist[-1]: return_message += "\n"
    return return_message

In [28]:
def Vectorize(tile_cat):
    #Resample classification 
    return_message = ''
    try:
        gscript.run_command('r.to.vect', flags='s', overwrite=True, quiet=True,
                            input='RF_fusion_tile_%s'%tile_cat, output='vect', type='area', column='class')
        #gscript.run_command('r.to.vect', overwrite=True, quiet=True,
                            #input='resamp_mmu', output='vect_staircase', type='area', column='class')
        return_message += "--> Vectorization succeeded."
    except:
        return_message += "ERROR: Vectorization of raster map failed for cutline '%s'. Please check for problem."%tile_cat
    return return_message   

def VectorSmooth(tile_cat, thresh_value="1000000", iter_value="2"):
    #Resample classification 
    return_message = ''
    try:
        gscript.run_command('v.generalize', overwrite=True, quiet=True, input='vect', type='area', output='vect_smooth', 
                            method='chaiken', threshold=thresh_value, iterations=iter_value)
        return_message += "--> Smoothing of vector layer succeeded."
    except:
        return_message += "ERROR: Smoothing of vector layer failed for cutline '%s'. Please check for problem."%tile_cat
    return return_message   

In [34]:
def ColorizeVector(tile_cat):
    #Declare empty string that will contain the messages to return
    return_message = ''
    try:
        gscript.run_command('v.colors', map='vect', use='attr',
                            column='class', rules=data['color_file'])
        gscript.run_command('v.colors', map='vect_staircase', use='attr',
                            column='class', rules=data['color_file'])
        gscript.run_command('v.colors', map='vect_smooth', use='attr',
                            column='class', rules=data['color_file'])
        return_message += "--> Application of colors table for vector layer succeeded."
    except:
        return_message += "ERROR: Application of colors table on vector failed for tile %s. Please check for problem."%tile_cat
    return return_message

In [25]:
def ExportRast(tile_cat, rasterlist):
    global list_rast_fusion
    #Declare empty string that will contain the messages to return
    return_message = ''
    for rastername in rasterlist:
        print rastername
        try:
        # Export raster
            export_dir = os.path.join(config_parameters['outputfolder_Raster'],rastername)
            #check_create_dir(export_dir)
            export_path = os.path.join(export_dir,"LC_%s_tile_%s.tif"%(rastername,tile_cat))
            gscript.run_command('r.out.gdal', quiet=True, overwrite=True, input=rastername, output=export_path,
                                format='GTiff', createopt='COMPRESS=DEFLATE') #Use flag c to not export colortable. Flag m to not export non-standard format of meta-data
            return_message += "--> Export of raster '%s' succeeded."%rastername
        except:
            return_message += "ERROR: Export of raster '%s' failed for cutline '%s'. Please check for problem."%(rastername,tile_cat)
        if rastername != rasterlist[-1]: return_message += "\n"
    return return_message

In [29]:
def ExportVect(tile_cat, vectlist):
    global list_rast_fusion
    #Declare empty string that will contain the messages to return
    return_message = ''
    for vectname in vectlist:
        print vectname
        try:
            # Export vector
            export_dir = os.path.join(config_parameters['outputfolder_Vecteur'],vectname)
            print export_dir
            #check_create_dir(export_dir, quiet=False)
            export_path = os.path.join(export_dir, "LC_%s_tile_%s.gpkg"%(vectname,tile_cat))
            print export_path
            gscript.run_command('v.out.ogr', quiet=False, overwrite=True,  input=vectname,
                                output=export_path, format='GPKG', flags='m') #Use flag c to not export colortable. Flag m to not export non-standard format of meta-data
            return_message += "--> Export of vector succeeded."
        except:
            return_message += "ERROR: Export of vector failed for cutline '%s'. Please check for problem."%tile_cat
    return return_message

In [30]:
def Clean(tile_cat):
    #Declare empty string that will contain the messages to return
    return_message = ''
    try:
        for layer in list_rast_fusion:
            gscript.run_command('g.remove', quiet=True, flags='f', type="raster", name=layer)
        return_message += "--> Mapset cleaned"
    except:
        return_message += "ERROR: during mapset cleaning. Please check for problem."
    return return_message

In [31]:
def Worker(tile_cat):
    import subprocess
    start_tile = start_processing() 
    print "Start processing on tile %s"%tile_cat
    #Declare empty list for saving output messages
    output_message = [] 
    
    # Launch mapset
    message = launch_mapset(tile_cat)  
    [output_message.append(a) for a in message]
    print "\n".join(message)

    # Allow access to other mapset 
    message = GetMapsetsAccess(["FUSIONS"])
    output_message.append(message)
    print message
    
    # Define computional region and mask
    message = DefineComputationRegionAndMask(tile_cat,resamp_resolution='1', grow_mmu='16')
    output_message.append(message)
    print message    

    # Resample
    #message = Resamp(tile_cat)
    #output_message.append(message)
    #print message 
    
    # Create forestry mask
    #message = GetForestryMask(tile_cat, schrink_growth_radius='15', overwrite=True)
    #output_message.append(message)
    #print message 

    # 1. Reclassification (removing MMU) according to rules 
    # MMU of 15m everywhere with special rule to prevent growing of buildings from small patches sharing the largest border with a building
    #message = ReclassAccordingToRule(tile_cat,
     #   ["CASE WHEN area < 15 AND label not in (11,18,19,21,26,28,29,31,38,39,58,59) THEN CASE WHEN first_label = 2 THEN second_label ELSE first_label END END",],
          #overwrite=True, intermediate=True)
    #output_message.append(message)
    #print message 
    
    #2. Reclassification (removing MMU) according to rules 
    # MMU of 500m in forest only for 8 and 9
    #message = ReclassAccordingToRule(tile_cat,
     #   ["CASE WHEN forest_mask = 1 AND area < 500 THEN CASE WHEN label in (8,9) THEN first_label ELSE label END END",],
      #  overwrite=True, intermediate=True)
    #output_message.append(message)
    #print message 
    
    # 3. Reclassification (removing MMU) according to rules 
    # MMU of 500m for the crops
    #message = ReclassAccordingToRule(tile_cat,
     #   ["CASE WHEN label = 6 AND area < 500 THEN CASE WHEN first_label = 2 THEN second_label ELSE first_label END END",],
      #  overwrite=True, intermediate=True)
    #output_message.append(message)
    #print message 
    
    # Reclassification (removing MMU) according to rules 
    # MMU of 500m for the crops
    #message = ReclassAccordingToRule(tile_cat,
     #   ["CASE WHEN label = 6 AND area < 500 THEN first_label ELSE label END",],
      #  overwrite=True, intermediate=True)
    #output_message.append(message)
    #print message 
    
    # 4. Reclassification (removing MMU) according to rules 
    # MMU de 200m pour les patches mixtes de sols nus et sols imperméabilisés
    #message = ReclassAccordingToRule(tile_cat,
    #    ["CASE WHEN label in (1,4) AND area < 200 THEN CASE WHEN first_label in (1,4) THEN first_label ELSE label END END",],
    #    overwrite=True, intermediate=False)
    #output_message.append(message)
    
    # Clip raster to match the extend of the initial tiles
    #message = ClipToTileBorder(tile_cat, ['resamp_mmu'])
    #output_message.append(message)
    #print message 

    # ColorizeRaster
    #message = ColorizeRaster(tile_cat, ['resamp_mmu'])
    #output_message.append(message)
#    print message
    
    # ExportRast 
    #message = ExportRast(tile_cat, ['resamp_mmu'])
    #output_message.append(message)
#    print message

    # Vectorize
    message = Vectorize(tile_cat)
    output_message.append(message)
    print message 

    # Vector - Smoothing
    #message = VectorSmooth(tile_cat, thresh_value="10000000", iter_value="3")
    #output_message.append(message)
    #print message 

    # Vector - Generalization
    #message = VectorSmooth()
    #output_message.append(message)
#    print message 

    # ColorizeVector
    #message = ColorizeVector(tile_cat)
    #output_message.append(message)
    #print message 

    # ExportVect 
    #message = ExportVect(tile_cat, ['vect','vect_smooth','vect_staircase'])
    message = ExportVect(tile_cat, ['vect'])
    output_message.append(message)
    print message

    # Clean 
    #message = Clean(tile_cat)
    #output_message.append(message)
    #print message

    #Print processing time
    message = print_processing_time(start_tile, "Prediction for tile '%s' achieved in "%tile_cat)
    output_message.append(message)
#    print message
    
    #Export Log file
    fout = open(os.path.join(config_parameters['outputfolder_Logfile'],"Log_Prediction_tile_%s.txt"%tile_cat),"w")
    [fout.writelines('%s\n'%content) for content in output_message]
    fout.close()

## Create new directories

In [62]:
# Check and create folder if needed
check_create_dir(config_parameters['outputfolder'])
check_create_dir(config_parameters['outputfolder_Logfile'])
check_create_dir(config_parameters['outputfolder_Raster'])
check_create_dir(config_parameters['outputfolder_Vecteur'])

The folder '/export/miro/cbassine/fusion/RF_fusion/Postprocess6' already exists
The folder '/export/miro/cbassine/fusion/RF_fusion/Postprocess6/Log_file' has been created
The folder '/export/miro/cbassine/fusion/RF_fusion/Postprocess6/Raster' already exists
The folder '/export/miro/cbassine/fusion/RF_fusion/Postprocess6/Vecteur' already exists


# Import fusions

In [63]:
run /export/miro/cbassine/fusion/RF_fusion/WALOUS-master/SRC/config_postprocess.py

In [64]:
# Create mapset 
launch_mapset("FUSIONS")

["Location 'WALOUS_31370' already exist",
 "'FUSIONS' mapset already exists in location 'WALOUS_31370'",
 "You are now working in mapset 'WALOUS_31370/FUSIONS'"]

In [75]:
# Create a list of paths to files with a specific name prefix
list_of_raster=[]
list_file = glob.glob(os.path.join(data['fusion_folder'][1],"RF_fusion_tile_*.tif"))
[list_of_raster.append((os.path.splitext(a)[0].split(os.sep)[-1],a)) for a in list_file]
print "There are %s paths in the list"%len(list_file)

There are 262 paths in the list


In [76]:
list_for_import=[]
list_file = glob.glob(os.path.join(data['fusion_folder'][1],"RF_fusion_tile_*.tif"))
#[list_for_import.append((os.path.splitext(a)[0].split(os.sep)[-1].split(os.sep)[-1])) for a in list_file]
[list_for_import.append((a[-20:-4]).split('_')[-1]) for a in list_file]
print list_for_import

['2', '10', '15', '3', '4', '8', '7', '14', '11', '17', '20', '16', '13', '0', '19', '6', '5', '18', '23', '24', '9', '12', '22', '25', '21', '30', '35', '26', '39', '31', '37', '304', '44', '43', '32', '28', '42', '47', '49', '38', '51', '56', '40', '52', '50', '57', '59', '45', '48', '46', '36', '54', '62', '58', '60', '65', '61', '72', '71', '68', '67', '66', '34', '79', '78', '69', '27', '76', '63', '84', '70', '74', '64', '77', '86', '73', '83', '80', '88', '91', '81', '89', '55', '82', '96', '95', '93', '97', '85', '100', '90', '87', '103', '101', '107', '94', '105', '306', '108', '114', '106', '113', '111', '99', '98', '117', '112', '110', '104', '109', '127', '122', '119', '123', '118', '126', '75', '121', '116', '129', '128', '124', '120', '309', '310', '311', '315', '313', '312', '319', '308', '115', '307', '323', '149', '151', '125', '152', '321', '155', '318', '324', '322', '161', '159', '316', '92', '314', '150', '166', '164', '165', '168', '317', '169', '162', '170', '158

In [68]:
import pickle

with open (config_parameters['list_tiles'],'rb') as outputfile:
    tile_aoi=pickle.load(outputfile)
tile_aoi

[u'1',
 u'2',
 u'3',
 u'4',
 u'5',
 u'6',
 u'7',
 u'8',
 u'9',
 u'10',
 u'11',
 u'12',
 u'13',
 u'14',
 u'15',
 u'16',
 u'17',
 u'18',
 u'19',
 u'20',
 u'21',
 u'22',
 u'23',
 u'24',
 u'25',
 u'26',
 u'27',
 u'28',
 u'29',
 u'30',
 u'31',
 u'32',
 u'33',
 u'34',
 u'35',
 u'36',
 u'37',
 u'38',
 u'39',
 u'40',
 u'41',
 u'42',
 u'43',
 u'44',
 u'45',
 u'46',
 u'47',
 u'48',
 u'49',
 u'50',
 u'51',
 u'52',
 u'53',
 u'54',
 u'55',
 u'56',
 u'57',
 u'58',
 u'59',
 u'60',
 u'61',
 u'62',
 u'63',
 u'64',
 u'65',
 u'66',
 u'67',
 u'68',
 u'69',
 u'70',
 u'71',
 u'72',
 u'73',
 u'74',
 u'75',
 u'76',
 u'77',
 u'78',
 u'79',
 u'80',
 u'81',
 u'82',
 u'83',
 u'84',
 u'85',
 u'86',
 u'87',
 u'88',
 u'89',
 u'90',
 u'91',
 u'92',
 u'93',
 u'94',
 u'95',
 u'96',
 u'97',
 u'98',
 u'99',
 u'100',
 u'101',
 u'102',
 u'103',
 u'104',
 u'105',
 u'106',
 u'107',
 u'108',
 u'109',
 u'110',
 u'111',
 u'112',
 u'113',
 u'114',
 u'115',
 u'116',
 u'117',
 u'118',
 u'119',
 u'120',
 u'121',
 u'122',
 u'123',
 

In [65]:
# Delete existing raster from previous versions processings
for rast in list_of_raster:
    gscript.run_command('g.remove', flags='f', type='raster', name=rast[0])

In [ ]:
# Import individual rasters (for each tile) --ERREUR
start_import = start_processing()
for rast in list_of_raster:
    gscript.run_command('r.in.gdal', overwrite=True, input=rast[1] , output=rast[0])
print_processing_time(start_import, "Import achieved in ")

In [79]:
# Create virtual raster
gscript.run_command('r.buildvrt', overwrite=True, 
                    input=",".join([a[0] for a in list_of_raster]), 
                    output='fusion_lc')

0

# Apply functions on each tile

In [77]:
# Set number of cores to use
ncores = 16

In [80]:
#list_cat=[]
#for i in range(800,810):
#    list_cat.append(str(i))
list_cat = ['300','301','302','303','304','305','306','307','308','309','310','311','312','313','314','315','316','317','318','319','320','321','322','323','324']

In [81]:
# Launch processes in parallel
start_parallel = start_processing()
p = Pool(ncores)
#output = p.map(Worker, tile_list[1:2])  # Launch the processes for as many items in the list (if function with a return, the returned results are ordered thanks to 'map' function)
output=p.map(Worker, list_cat[:])
p.close()
p.join()
# Print
print_processing_time(start_parallel, "Computation (on %s cores) achieved in "%ncores)

Start processing on tile 300
Start processing on tile 301
Start processing on tile 302
Start processing on tile 305
Start processing on tile 306
Start processing on tile 303
Start processing on tile 307
Start processing on tile 308
Start processing on tile 310
Start processing on tile 312
Start processing on tile 311
Start processing on tile 304
Start processing on tile 314
Start processing on tile 315
Start processing on tile 309
Start processing on tile 313
Location 'WALOUS_31370' already exist
'302' mapset already exists in location 'WALOUS_31370'
You are now working in mapset 'WALOUS_31370/302'Location 'WALOUS_31370' already exist
'301' mapset already exists in location 'WALOUS_31370'
You are now working in mapset 'WALOUS_31370/301'

Location 'WALOUS_31370' already exist
'305' mapset already exists in location 'WALOUS_31370'
You are now working in mapset 'WALOUS_31370/305'Location 'WALOUS_31370' already exist
'300' mapset already exists in location 'WALOUS_31370'
You are now workin

You are now working in mapset 'WALOUS_31370/318'
Access to other mapset added
Working on tile '318'
--> Setting of computational region and masks succeeded.
--> Export of vector succeeded.
Start processing on tile 319
Location 'WALOUS_31370' already exist
'319' mapset already exists in location 'WALOUS_31370'
You are now working in mapset 'WALOUS_31370/319'
Access to other mapset added
Working on tile '319'
--> Setting of computational region and masks succeeded.
--> Vectorization succeeded.
vect
/export/miro/cbassine/fusion/RF_fusion/Postprocess6/Vecteur/vect
/export/miro/cbassine/fusion/RF_fusion/Postprocess6/Vecteur/vect/LC_vect_tile_318.gpkg
--> Export of vector succeeded.
Start processing on tile 320
Location 'WALOUS_31370' already exist
'320' mapset already exists in location 'WALOUS_31370'
You are now working in mapset 'WALOUS_31370/320'
Access to other mapset added
--> Vectorization succeeded.
vect
/export/miro/cbassine/fusion/RF_fusion/Postprocess6/Vecteur/vect
/export/miro/cb

'Computation (on 16 cores) achieved in 36 minutes and 24.8 seconds'

**Check log file for ERRORS**

In [115]:
# Get list of csv with classification feature of individual tiles
import glob
list_log = glob.glob(os.path.join(config_parameters['outputfolder_Logfile'],"Log_Prediction_tile_*.txt"))
print "%s log files in the folder"%len(list_log)

810 log files in the folder


In [116]:
# Declare new counter
count = 0
# Declare new list that will contain list of tile with error
tile_error_list = []
# Loop on list of log file
for logfile in list_log:
    got_error = False
    tile_num = os.path.splitext(os.path.basename(logfile))[0].split("_")[-1]
    fin = open(logfile, 'r')
    for row in fin:
        if row[:5] == "ERROR":  # If at least one line have error message, the whole file will be counted as 1 error
            got_error = True
    if got_error:    
        count += 1
        tile_error_list.append(tile_num)  # Add tile number to the list
# Print
print "%s tile(s) faced an ERROR during the processing.\n"%count

# Update tile list with only tiles that have ERROR in log 
print "\n".join(["Error on tile %s"%(a) for a in tile_error_list])

374 tile(s) faced an ERROR during the processing.

Error on tile 3
Error on tile 10
Error on tile 9
Error on tile 6
Error on tile 7
Error on tile 8
Error on tile 4
Error on tile 5
Error on tile 1
Error on tile 144
Error on tile 145
Error on tile 146
Error on tile 147
Error on tile 148
Error on tile 64
Error on tile 128
Error on tile 149
Error on tile 16
Error on tile 65
Error on tile 32
Error on tile 160
Error on tile 208
Error on tile 48
Error on tile 112
Error on tile 129
Error on tile 130
Error on tile 150
Error on tile 80
Error on tile 131
Error on tile 224
Error on tile 132
Error on tile 133
Error on tile 134
Error on tile 135
Error on tile 240
Error on tile 136
Error on tile 137
Error on tile 138
Error on tile 17
Error on tile 139
Error on tile 140
Error on tile 33
Error on tile 141
Error on tile 142
Error on tile 143
Error on tile 151
Error on tile 161
Error on tile 225
Error on tile 209
Error on tile 66
Error on tile 81
Error on tile 152
Error on tile 49
Error on tile 34
Error 

**-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-**

# Create VRT with all raster products

In [ ]:
# Create mapset 
launch_mapset("POSTPROCESS")

In [ ]:
# Add access to other mapsets
GetMapsetsAccess(["%s"%cat for cat in list_cat])

In [ ]:
# Generate VRT
gscript.run_command('r.buildvrt', overwrite=True, 
                    input=",".join(["forestmask_binary@%s"%cat for cat in list_cat]), 
                    output='forestmask_binary_vrt')

In [ ]:
# Generate VRT
gscript.run_command('r.buildvrt', overwrite=True, 
                    input=",".join(["resamp@%s"%cat for cat in list_cat]), 
                    output='resamp_vrt')

In [ ]:
# Generate VRT
gscript.run_command('r.buildvrt', overwrite=True, 
                    input=",".join(["reclass_tmp@%s"%cat for cat in list_cat]), 
                    output='reclass_tmp_vrt')

In [ ]:
# Generate VRT
gscript.run_command('r.buildvrt', overwrite=True, 
                    input=",".join(["resamp_mmu@%s"%cat for cat in list_cat]), 
                    output='resamp_mmu_vrt')

**-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-**

# Extract vectorial outputs at the 'commune' level

## Import layers for the whole wallonia

In [ ]:
run ../SRC/config_postprocess.py

In [ ]:
# Create mapset 
launch_mapset("PERMANENT")

**Communes**

In [ ]:
# Import vector layer of the belgian communes
gscript.run_command('v.import', overwrite=True, 
                    epsg='31370', input=data['communes'][1], output=data['communes'][0])

In [ ]:
# Get list of communes ID
list_com = [code 
            for code 
            in gscript.read_command('v.db.select', flags='c', map=data['communes'][0], columns='cd_munty_r').split('\n') 
            if len(code)>0]

**Cadastral blocks**

In [ ]:
# Import vector layer of the belgian communes
gscript.run_command('v.import', overwrite=True, 
                    epsg='3812', input=data['CaPa'][1], output=data['CaPa'][0])

## Extract LC proportion by CaPa for each commune

In [ ]:
# List of communes ID intersecting MARCHE area
list_com = ['61012','83012','83028','83034','83040','91030','91064','91120']

In [ ]:
# Define region based on the 1m spatial resolution raster LC output
g.region raster=clip_output

In [ ]:
v.to.rast --overwrite input=CaPa output=CaPa use=attr attribute_column=cat label_column=CaPaKey memory=2000

# Extract polygons for each 'commune'

In [ ]:
v.db.addcolumn map=test_overlay@3219 columns="class integer"

In [ ]:
v.db.update map=test_overlay@3219 column=class query_column=a_class where="b_class is null"

In [ ]:
v.db.update map=test_overlay@3219 column=class query_column=a_class where="(a_class is not null) AND (b_class is not null) "

In [ ]:
v.dissolve input=test_overlay@3219 column=class output=test_dissolve